In [12]:
import pandas as pd
import transformers
import torch
from tqdm import tqdm
from datasets import Dataset

In [13]:
QA_o = pd.read_csv('QA_open.csv')
QA_o

,questionType,asin,answerTime,unixTime,question,answerType,answer,__index_level_0__
0,open-ended,0972683275,"Dec 3, 2013",1.386058e+09,how far out does the arm extend?,NaN,18 inches on our TV.,10
1,open-ended,0972683275,"Nov 6, 2013",1.383725e+09,"Will it mount a 32"" Vizio? The bolts in back a...",NaN,Is should fit. If necessary you can use the 4 ...,17
2,open-ended,1400532620,"Oct 26, 2010",1.288076e+09,why is it more expensive here than direct from...,NaN,It looks like the one sold here through Amazon...,31
3,open-ended,1400699169,"Oct 8, 2013",1.381216e+09,How easy is this to use in direct sunlight? gl...,NaN,"No problem, you can adjust the light.",94
4,open-ended,1400699169,"Dec 16, 2013",1.387181e+09,does it come with a power cord? anything else?,NaN,Yes it came with the power cord the wall adapt...,97
...,...,...,...,...,...,...,...,...
38890,open-ended,B00L26YDA4,"Jul 10, 2014",1.404976e+09,What happened to the DLNA support claimed in t...,NaN,Dlna works.,314229
38891,open-ended,B00L26YDA4,"Aug 24, 2014",1.408864e+09,When will the firmware update that allows rena...,NaN,Hi Justin: Sabrent just released a new firmwar...,314230
38892,open-ended,B00L26YDA4,"Aug 21, 2014",1.408604e+09,Can I use AirPlay to connect to two of these a...,NaN,I don't think your second question was answere...,314232
38893,open-ended,B00L26YDA4,"Nov 10, 2014",1.415606e+09,I keep getting message when trying to connect ...,NaN,I noticed that the most recent firmware update...,314234


In [16]:
QA_o = Dataset.from_pandas(QA_o)
QA_o

Dataset({
    features: ['questionType', 'asin', 'answerTime', 'unixTime', 'question', 'answerType', 'answer', '__index_level_0__'],
    num_rows: 38895
})

In [19]:
QA_o_f = []
QA_o_r = []

In [24]:
from groq import Groq

for sample in tqdm(QA_o, desc="Processing samples"):
    
    question = sample['question']
    answer = sample['answer']
    client = Groq(api_key = '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')
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": "Your role is to judge whether the answer is objective to the question. You must answer YES or NO"
            },
            {
                "role": "user",
                "content": f"question: {question}\n answer: {answer}"
            }
        ]
    )
    
    result = completion.choices[0].message.content
    if 'yes' in result.lower():
        QA_o_f.append(sample)
    else:
        QA_o_r.append(sample)

Processing samples:  12%|█▏        | 4498/38895 [3:24:29<26:03:47,  2.73s/it] 


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01j8j24e74eww941kb13gmn5yd` on : Limit 500000, Used 499981, Requested 469. Please try again in 1m17.732599999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [27]:
QA_o_f_df = pd.DataFrame(QA_o_f)
QA_o_f_df.to_csv("QA_o_filtered.csv", index=False)
QA_o_f_df

,questionType,asin,answerTime,unixTime,question,answerType,answer,__index_level_0__
0,open-ended,1400532655,"Dec 24, 2012",1.356336e+09,can you use an Amazon gift card to purchase bo...,None,Amazon cards cannot be used.,106
1,open-ended,1400532655,"Dec 6, 2013",1.386317e+09,facebook: can you get facebook on the nook tablet,None,"Yes. However, Facebook games can't be played o...",109
2,open-ended,1400599997,"Aug 10, 2011",1.312960e+09,"is the Nook 3g Free ? Is the Nook 3g free, lik...",None,"Hi Mark, The Nook 3g is free and it's powered ...",122
3,open-ended,1400501466,"Nov 25, 2014",1.416902e+09,dose it have a talking GPS,None,No,126
4,open-ended,9983891212,"Nov 29, 2014",1.417248e+09,Is this only for cameras? Can it be used for a...,None,Yes,264
...,...,...,...,...,...,...,...,...
1334,open-ended,B000JMJWV2,"Oct 1, 2008",1.222844e+09,"how do you format it to your camera? So, how d...",None,usually you just go under playback or wherever...,23173
1335,open-ended,B000JO3N3S,"Jul 14, 2008",1.216019e+09,Can you put CD's onto an iPod without having t...,None,Yes you can add your CD's to your IPOD without...,23212
1336,open-ended,B000JO3N3S,"Nov 26, 2007",1.196064e+09,Can you put music on from CD's and not just it...,None,iPods can play a variety of different media ty...,23216
1337,open-ended,B000JNYWBG,"Sep 6, 2007",1.189062e+09,Only 16GB? Compared to current generation iPod...,None,"It's flash memory, not a hard drive. Flash is ...",23222


In [29]:
QA_o_r_df = pd.DataFrame(QA_o_r)
QA_o_r_df.to_csv("QA_o_removed.csv", index=False)
QA_o_r_df

,questionType,asin,answerTime,unixTime,question,answerType,answer,__index_level_0__
0,open-ended,0972683275,"Dec 3, 2013",1.386058e+09,how far out does the arm extend?,None,18 inches on our TV.,10
1,open-ended,0972683275,"Nov 6, 2013",1.383725e+09,"Will it mount a 32"" Vizio? The bolts in back a...",None,Is should fit. If necessary you can use the 4 ...,17
2,open-ended,1400532620,"Oct 26, 2010",1.288076e+09,why is it more expensive here than direct from...,None,It looks like the one sold here through Amazon...,31
3,open-ended,1400699169,"Oct 8, 2013",1.381216e+09,How easy is this to use in direct sunlight? gl...,None,"No problem, you can adjust the light.",94
4,open-ended,1400699169,"Dec 16, 2013",1.387181e+09,does it come with a power cord? anything else?,None,Yes it came with the power cord the wall adapt...,97
...,...,...,...,...,...,...,...,...
3154,open-ended,B000JNYWBG,"Jan 16, 2008",1.200470e+09,Messengers on the Itouch 16 Gb??? Do anyone kn...,None,yes if you have a jailbreaked touch and offici...,23226
3155,open-ended,B000JNYWBG,"Jan 15, 2008",1.200384e+09,New iPod touch Applications: What do you guys ...,None,"I am ready to by my ipod touch, but I can wait...",23227
3156,open-ended,B000JNYWBG,"Jan 7, 2008",1.199693e+09,Downloading Album Covers- How do I do this? Ho...,None,Some questions. I assume from what you are say...,23228
3157,open-ended,B000JNYWBG,"Dec 12, 2007",1.197446e+09,Downloading TV shows to iTouch: Can anyone tel...,None,I'm a big fan of my iPod touch as well (in fac...,23229
